# Advanced Topics : Asynchronous Programming

____

1. Generics
2. Delegates
3. Lambda Expressions
4. Events
5. Extension Methods
6. LINQ
7. Nulllable Types
8. Dynamic
9. Exception Handling
10. Asynchronous Programming



In [1]:
using System;
// using System.Collections.Generic;
// using System.Linq; 
// using System.Text;
// using System.Threading.Tasks;


The below script needs to be able to find the current output cell; this is an easy method to get it.

## Asynchronous Programming

____

The main essence

1. 


In Asynchronous programming, is when a function is called, program execution continues  to the next line, WITHOUT waiting for the function to complete.


It does not cause blocking. What are blocking operations

- Accessing the web
- Accessing database
- Accessing files
- Working with images


Traditonally in .NET we did the following

- Multi-Threading (complex to learn)
- Callbacks methods (complex to learn)

With .NET 4.5 and above, we use 

- Async / Await

It is called task based asynchronous model, adn we use async and await keywords.

Demonstration using  WPF, not be able to do it here, but we will substitute some code here, just to get the point across...

 
WE want to download contents from website, we use the following functions


1. DownloadHtml method, calls Point 2.
2. Call DownloadString method, takes a url and return html.
3. And then write the contents to file. 


Lets get a function to Download html file/contents:

```
public void DownloadHtml (string url){

    var webClient = new WebClient();
    var html = webClient.DownloadString(url)
    
    using(var streamWriter = new StreamWriter("url_contents")){
    
        streamWriter.Write(html);
    
    
    )
}

```

Then we call the method DownloadHtml method and pass a url

```
DownloadHtml("https://msdn.microsoft.com");

```

If we run run code there is a delay...and the app will hang.


In [2]:
using System.IO;
using System.Net;

In [3]:
public void DownloadHtml (string url){

    var webClient = new WebClient();
    var html = webClient.DownloadString(url);
    
    using(var streamWriter = new StreamWriter("url_contents")){
    
        streamWriter.Write(html);
    
    
    }
}


In [4]:
//DownloadHtml("https://shuttleonkent.co.za");
DownloadHtml("https://msdn.microsoft.com");

## How to make this an asynchronous implementation

____

All methods must have async keyword, the return type must be a Task type. What is this? Its an object that encapsulates the state of an asynchronous operation. It comes in a non-generic (for void) and generic form (for data types)

Syntax


1. public 
2. async
3. Task
4. Name of method
5. Add Sync Suffix
```
public async Task DownloadHtmlAsync (string url){

}

```

Now we need to determine, what blocking objects do we have
1. DownloadString
2. streamWriter.Write

These are all blocking method, but .NET gives us the non-blocking or asynchronous version of these methods, with the ```TaskAsync```  or ```Async``` Suffix after the methods.


By applying TaskAsync to ```webClient.DownloadStringTaskAync(url);```, instead of returning a string now, it will return a ```Task<string>```, and when we call TaskAsync method of the synchronous mehod version, then we need to add the word ```await``` in front of the method.


```
public async Task DownloadHtmlAsync (string url){

    var webClient = new WebClient();
    var html = await webClient.DownloadStringTaskAync(url);
    
    using(var streamWriter = new StreamWriter("url_contents")){ 
    
        await streamWriter.WriteAsync(html);
    
    }
}

```

What does the above mean? if we add ```await``` keyword, we may think that we need to wait for the method finish, since we have "wait" word.. sure w do...but not so quick, the ```await``` keyword its just marker to the compiler, that this method is a slow coach, and the thread will not be blocked, and control is immediately returned to caller of this method (DownloadHtml("https://msdn.microsoft.com")

Note that the statements below, this statement are not executed, rather control goes back to the calling method, then what is following that statement will be executed.

So ```using()```, will not be executed...but the DownloadHtmlAsync method will be executed. Only when await is finished does it come back to execute  ```using()```

On .NET we dont have to worry about a ```callback function```, that is ```using```, the commpiler does this work for us (we just need to use the async and await keywords).

We have another blocking method. (```write```), So we need to  update this. (see above)

In [5]:
using System;
using System.IO;
using System.Net;

In [6]:
public async Task DownloadHtmlAsync (string url){
    

    var webClient = new WebClient();
    var html = await webClient.DownloadStringTaskAsync(url);
                            
    Console.WriteLine("After webClient.DownloadStringTaskAync"); //not displayed??
    
    using(var streamWriter = new StreamWriter("url_contents.txt")){ 
        
       Console.WriteLine("Inside using"); 
       await streamWriter.WriteAsync(html);
    
    }
}

In [7]:
//DownloadHtmlAsync("https://phptravels.com/contact-us/");
DownloadHtmlAsync("https://msdn.microsoft.com");
Console.WriteLine("After DownloadHtmlAsync");

After DownloadHtmlAsync


## Now lets return a string...```Task<string>```

We will be going to return an ```await``` result, so they syntax changes a bit. So we create a method where we retutn an await result

```
public async Task<string> GetHtMLAsync(string url){
    var webClient = new WebClient();
    return  await webClient.DownloadStringTaskAsync(url);
}
```

When calling this method, since we returning an await, the calling method also need to add the await keyword.


```
var html = GetHtMLAsync("https://msdn.microsoft.com");
Console.WriteLine("HTML: {0}", html.Substring(0, 100));

(3,37): error CS1061: 'Task<string>' does not contain a definition for 'Substring' and no accessible extension method 'Substring' accepting a first argument of type 'Task<string>' could be found (are you missing a using directive or an assembly reference?)

```

The above returns a ```Task<string>``` state, so to get a string, we need to add ```await``` keyword.
    
```
var html = await GetHtMLAsync("https://msdn.microsoft.com");
Console.WriteLine("After GetHtML");
Console.WriteLine("HTML: {0}", html.Substring(0, 10));

```

e.g. the following will happen

1. var html = await GetHtMLAsync("https://msdn.microsoft.com");
2. waiting....
3. waiting....
4. waiting....
5. waiting....
6. Console.WriteLine("After GetHtML");
7. Console.WriteLine("HTML: {0}", html.Substring(0, 10));

or can also  just get the task, and then we can await.

```
var htmlTask = GetHtMLAsync("https://msdn.microsoft.com");
Console.WriteLine("After htmlTask");
var html = await htmlTask;
Console.WriteLine("After GetHtML");
Console.WriteLine("HTML: {0}", html.Substring(0, 10));
```

What the above proves that you dont have to add and ```await``` for an asynchronous operation. If you do not add it, then you can got ahead and perform other work, and you dont have any stopping of permformance. But as soon as you add, the ```await``` keyword, then you will wait, and control will reur

e.g. the following will happen

1. var htmlTask = GetHtMLAsync("https://msdn.microsoft.com");
2. Console.WriteLine("After htmlTask");
3. Console.WriteLine("After htmlTask");
4. Console.WriteLine("After htmlTask1");
5. Console.WriteLine("After htmlTask2");
6. var html = await htmlTask;
7. waiting....
8. waiting....
9. waiting....
10. waiting....
11. Console.WriteLine("After GetHtML");
12. Console.WriteLine("HTML: {0}", html.Substring(0, 10));


In [8]:
using System;
using System.IO;
using System.Net;

In [9]:
public async Task<string> GetHtMLAsync(string url){
    var webClient = new WebClient();
    return  await webClient.DownloadStringTaskAsync(url);
}

In [10]:
var html = await GetHtMLAsync("https://msdn.microsoft.com");
Console.WriteLine("After GetHtML");
Console.WriteLine("HTML: {0}", html.Substring(0, 10));


After webClient.DownloadStringTaskAync
Inside using
After GetHtML
HTML: <!DOCTYPE 


In [11]:
var htmlTask = GetHtMLAsync("https://msdn.microsoft.com");
Console.WriteLine("After htmlTask");
Console.WriteLine("After htmlTask1");
Console.WriteLine("After htmlTask2");
var html = await htmlTask;
Console.WriteLine("After GetHtML");
Console.WriteLine("HTML: {0}", html.Substring(0, 10));

After htmlTask
After htmlTask1
After htmlTask2
After GetHtML
HTML: <!DOCTYPE 
